<img src="./logo.jpg" alt="Jupyter_logo" class="img-rounded img-responsive" height="60" width="120" align="left">
<span><br><br><br /></span>
<br />
<h1 align='center' style='margin-bottom: 0px'>Scale data check Summary</h1>
<h4 align='center' style='margin-top: 0px'> SimingYan</h4>


This <span style="color:rgba(255,100,0,1)"><b>Jupyter</b></span> notebook <code>'check_ou_bdp.ipynb'</code> checks data from <b> ou to bdp </b>. file path is Q:\SHA_KQ\Digital Intelligence\siming_yan\check_ou_bdp.ipynb

<span style="font-size: small">* Notebook is writen in Python 3.8</span>

In [2]:
 

import pandas as pd
import numpy as np
import os
os.getcwd()
# import dt
import re
from datetime import datetime
#  wms_warehouse_name,wms_company_name,asn_id,internal_id,
# closed_date, receive_qty,original_qty,receive_weight
 

#  wms_warehouse_name,wms_company_name,order_id,shipment_id,
# closed_date, shipped_qty,original_order_qty,shipped_weight

import jupyterthemes
from jupyterthemes import jtplot

jtplot.style(theme='onedork')

# set "context" (paper, notebook, talk, poster)
# scale font-size of ticklabels, legend, etc.
# remove spines from x and y axes and make grid dashed
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')
  

In [12]:
def date_pre(df, col_name):
    """
    愚蠢的写法, 从 2021-06-03 12:43:13.487 转换到20210603 
    """
    for i in col_name:
        df.loc[:,i] = df[i].astype(str)
        df.loc[:,i] = df[i].str.slice(0,10)
        df.loc[:,i] = [j.replace('-','') for j in df[i]]
    # df = df.dropna(subset = col_name)
    return df.head(1)

def rename_dataf(ou_df, close_date, id, line_id, qty):
    """
    对ou的data做统一性重命名处理, 只取4列 ['close_date', 'id', 'line_id', 'qty']
    """
    ou_df = ou_df[[close_date, id, line_id, qty]]
    ou_df.columns = ['close_date', 'id', 'line_id','qty']
    return ou_df
    


def compare_data(df1, df2, date1, date2, col1_line, col2_line, col1_quant, col2_quant):
    """
    数据对比函数, 输出两个df, 一个为全部的对比值, 另一个为有差异的subset
    两个df对比(先写bdp的数据), 两个时间戳, 两个line字段名称, 两个quant字段名称
    """
    ff1 = []
    ff2 = []
    ff3 = []
    def make_df(df):
        df = pd.DataFrame(df).reset_index()
        df.columns = ['date_time', 'quant']
        return df
    ff1 = make_df(df1.groupby([date1])[col1_line].nunique())
    ff2 = make_df(df2.groupby([date2])[col2_line].nunique())
    
    ff1 = ff1.merge(ff2, on = 'date_time', how = 'outer')

    ff2 = make_df(df1.groupby([date1])[col1_quant].sum()).merge(
        make_df(df2.groupby([date2])[col2_quant].sum()),
        on = 'date_time',
        how = 'outer'
    )
    ff1 = ff1.merge(ff2, on = 'date_time', how = 'outer')
    ff1.columns = ['date_time', 'bdp_hdr_ct', 'ou_hdr_ct', 'bdp_qty', 'ou_qty']
    """
    数据差异subset
    """
    ff2 = ff1[ff1['bdp_qty'] - ff1['ou_qty'] != 0].dropna(how = 'any')
    return ff1, ff2

def colname_bdp(df):
    re1 = re.compile(r'(?<=\.).+')
    df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
    return df.head(3)

def detail_check(inb_bose_bdp, inb_bose_oud, wrong_time, cols_bdp, cols_ou):
    """
    输入两个表名字后, 写有问题的时间查看细节区别,
    cols = id, qty, date
    """
    df_detail = pd.DataFrame(
        inb_bose_bdp[inb_bose_bdp[cols_bdp[2]] == wrong_time][
            cols_bdp
            ].groupby(cols_bdp[0])[cols_bdp[1]].sum()
            ).reset_index().merge(
                pd.DataFrame(inb_bose_oud[inb_bose_oud[cols_ou[2]] == wrong_time][
                    cols_ou
                    ].groupby(cols_ou[0])[cols_ou[1]].sum()).reset_index(),
    left_on = cols_bdp[0], right_on = cols_ou[0], how = 'outer'
)   
    df_detail.columns = ['bdp_id','bdp_qty','ou_id','ou_qty']
    df_detail2 = df_detail[df_detail['bdp_qty'] - df_detail['ou_qty'] != 0]

    return df_detail,df_detail2





---
<span style="color:rgba(255,244,0,1);font-size:42px"><b>BOSE_SH</b></span> 

> inbound

In [58]:
## bdp dwd final table.
inb_bose_bdp = pd.read_csv('./bose_in_jun_bdp.csv')
# outb_0601_bose = pd.read_csv('./outb_0601_bos.csv')
## 业务数据
# bos_jun_in_ou = pd.read_excel("./Bose June Inbound.xls")
inb_bose_oud = pd.read_excel("./Bose June Inbound.xls")
inb_bose_oud = inb_bose_oud.dropna(subset = ['Close Date'])

# bos_jun_ou_ou['SHIP_DATE_TIME']+ pd.Timedelta(hours=8) 
# 不应该加8, 看样子ou自己加过了.
# bos_jun_ou_ou['SHIP_DATE_TIME'] = bos_jun_ou_ou['SHIP_DATE_TIME'] + pd.Timedelta(hours=8) 

In [59]:
inb_bose_bdp.columns

Index(['wms_company_id', 'wms_warehouse_id', 'internal_id', 'asn_id',
       'asn_line_id', 'original_qty', 'receive_qty', 'create_time',
       'closed_date', 'receipt_time'],
      dtype='object')

In [60]:
"""
筛选条件, creat time <= 6-30
"""
inb_bose_oud['Creation Date'].max()
inb_bose_bdp = inb_bose_bdp[inb_bose_bdp['create_time'] < '2021-07-01 00:00:01']


In [97]:
# """
# 筛选条件2, creat time <= 6-30
# """
# # inb_bose_oud['Creation Date'].max()
# inb_bose_oud[inb_bose_oud['id'].str.extract('(\d{0,8})').isin(
#     inb_bose_oud['close_date'].astype(object))].dropna()
# inb_bose_oud[inb_bose_oud['id'].str.extract('(\d{0,8})').isin(inb_bose_oud['close_date'])].dropna()
# inb_bose_oud['id'][inb_bose_oud['id'].str.extract('(\d{0,8})')[0] == '20210608']

In [61]:
date_pre(inb_bose_oud, ['Close Date'])
date_pre(inb_bose_bdp, ['closed_date'])
inb_bose_oud = rename_dataf(inb_bose_oud, 'Close Date',  'Receipt ID', 'Total Lines', 'Total Quantity')

In [62]:
# line count
line_df = pd.concat([pd.DataFrame(inb_bose_bdp.groupby(
    'closed_date')['asn_line_id'].count()
    ).reset_index(),
    pd.DataFrame(
    inb_bose_oud.groupby('close_date')['line_id'].sum()
    ).reset_index()],
    axis = 1)
line_df = line_df.drop('close_date', axis = 1)
line_df.columns = ['date_time', 'bdp_line_ct', 'ou_line_ct']
line_df=line_df.dropna(subset = ['ou_line_ct'])

In [63]:
# 
bose_214_in, bose_214_in_diff = compare_data(
    inb_bose_oud,inb_bose_bdp, 
    'close_date', 'closed_date', 
    'id','asn_id',
    'qty','receive_qty'
);



> summary of inbound line. check header num, line num and qty. 

In [64]:
bose_in_summary = bose_214_in.dropna(
    subset =['bdp_hdr_ct']
    ).merge(
        line_df, on = 'date_time'
        );bose_in_summary

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty,bdp_line_ct,ou_line_ct
0,20210601,6.0,6,60077.0,60077.0,31,31.0
1,20210602,8.0,8,15436.0,15436.0,145,145.0
2,20210603,24.0,24,33955.0,33955.0,676,676.0
3,20210604,17.0,17,20885.0,20885.0,635,635.0
4,20210605,3.0,3,2277.0,2277.0,3,3.0
5,20210607,3.0,3,4684.0,4684.0,18,18.0
6,20210608,5.0,5,8539.0,8521.0,87,87.0
7,20210609,20.0,20,88250.0,88250.0,124,124.0
8,20210610,13.0,13,43915.0,43915.0,43,43.0
9,20210611,10.0,10,6888.0,6888.0,58,58.0


> days ou and bdp have differences

In [65]:
bose_214_in_diff

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
6,20210608,5.0,5,8539.0,8521.0
12,20210615,5.0,5,11599.0,10759.0
20,20210623,20.0,20,51222.0,50253.0
27,20210630,19.0,21,47622.0,47598.0


In [66]:
line_df_diff = line_df[line_df['bdp_line_ct'] - line_df['ou_line_ct'] != 0];line_df_diff

,date_time,bdp_line_ct,ou_line_ct
27,20210630,155,153.0


--- 

### detail_check on single order

In [26]:
def detail_check(inb_bose_bdp, inb_bose_oud, wrong_time, cols_bdp, cols_ou):
    """
    输入两个表名字后, 写有问题的时间查看细节区别,
    cols = id, qty, date
    """
    df_detail = pd.DataFrame(
        inb_bose_bdp[inb_bose_bdp[cols_bdp[2]] == wrong_time][
            cols_bdp
            ].groupby(cols_bdp[0])[cols_bdp[1]].sum()
            ).reset_index().merge(
                pd.DataFrame(inb_bose_oud[inb_bose_oud[cols_ou[2]] == wrong_time][
                    cols_ou
                    ].groupby(cols_ou[0])[cols_ou[1]].sum()).reset_index(),
    left_on = cols_bdp[0], right_on = cols_ou[0], how = 'outer'
)   
    df_detail.columns = ['bdp_id','bdp_qty','ou_id','ou_qty']
    df_detail2 = df_detail[df_detail['bdp_qty'] - df_detail['ou_qty'] != 0]

    return df_detail,df_detail2



In [69]:
detail_1, detail2 = detail_check(inb_bose_bdp, inb_bose_oud, '20210608',
        ['asn_id', 'receive_qty', 'closed_date'],
        ['id', 'qty', 'close_date']
        )

> '20210608' differences

In [70]:
detail_1

,bdp_id,bdp_qty,ou_id,ou_qty
0,1021099395,1152.0,1021099395,1152
1,2021060801,18.0,2021060801,36
2,A20210607,919.0,A20210607,919
3,A20210608,288.0,A20210608,288
4,S20210608,6144.0,S20210608,6144


In [351]:
detail2

# 这个料有两个客户料号

,bdp_id,bdp_qty,ou_id,ou_qty
1,2021060801,18.00,2021060801,36


In [336]:
inb_bose_oud[inb_bose_oud['id'] == '2021060801']

,close_date,id,line_id,qty
1534,20210608,2021060801,1,36


---
## bose outbound check 

In [98]:
## bdp dwd final table.
# outb_0601_bose = pd.read_csv('./jun_bose_out.csv')
outb_0601_bose = pd.read_csv('./bose_out_test.csv')
bos_jun_ou_ou = pd.read_excel("./Bose June Outbound.xls", header = 1)
bos_jun_ou_ou = bos_jun_ou_ou.dropna(subset = ['SHIP_DATE_TIME'])
 

In [99]:
outb_0601_bose.head()

,wms_company_id,wms_warehouse_id,wms_warehouse_name,wms_warehouse_id.1,order_id,shipment_id,order_line,original_order_qty,shipped_qty,create_date,closed_date
0,CE,BOSE_SH,BOSE_SH,BOSE_SH,0324464528,286972.0,848642.0,1.0,1.0,2021-05-31 02:06:02,2021-06-01 19:29:15
1,CE,BOSE_SH,BOSE_SH,BOSE_SH,0324464592,286987.0,848672.0,1.0,1.0,2021-05-31 05:06:13,2021-06-01 19:29:16
2,CE,BOSE_SH,BOSE_SH,BOSE_SH,0324464599,286990.0,848687.0,1.0,1.0,2021-05-31 05:06:16,2021-06-01 19:29:16
3,CE,BOSE_SH,BOSE_SH,BOSE_SH,0324465003,286975.0,848645.0,1.0,1.0,2021-05-31 02:06:02,2021-06-01 19:29:15
4,CE,BOSE_SH,BOSE_SH,BOSE_SH,0324465004,286973.0,848643.0,1.0,1.0,2021-05-31 02:06:02,2021-06-01 19:29:15


In [100]:
outb_0601_bose[outb_0601_bose['create_date'] < '2021-06-01'].max()
## very important line 
outb_0601_bose = outb_0601_bose[outb_0601_bose['create_date'] > '2021-05-31 17:59:59']
##

In [101]:
date_pre(outb_0601_bose, ['closed_date'])
date_pre(bos_jun_ou_ou,  ['SHIP_DATE_TIME'])

,CREATE_DATE_TIME,COMPANY,SHIP_TO,SHIP_TO_NAME,SHIPMENT_ID,CUSTOMER PO,Customer Part Number,ITEM,TOTAL\nQTY,SALES_UNIT,DIVISION,ORDER_TYPE,Inventory Status,PCKDTE,SHIP_DATE_TIME,SHIP_STS,Sales_office,SF_Tracking_num
0,2021-06-01 02:46:02.010000,CE,0125815743,凯 张,0324472164,N056119789,NaN,834167-0300,1.0,EA,NaN,ZLF1,1000.0,2021-06-01 12:44:27.907,20210601,Closed,4008,NaN


In [102]:
[outb_0601_bose.columns, bos_jun_ou_ou.columns]

[Index(['wms_company_id', 'wms_warehouse_id', 'wms_warehouse_name',
        'wms_warehouse_id.1', 'order_id', 'shipment_id', 'order_line',
        'original_order_qty', 'shipped_qty', 'create_date', 'closed_date'],
       dtype='object'),
 Index(['CREATE_DATE_TIME', 'COMPANY', 'SHIP_TO', 'SHIP_TO_NAME', 'SHIPMENT_ID',
        'CUSTOMER PO', 'Customer Part Number', 'ITEM', 'TOTAL\nQTY',
        'SALES_UNIT', 'DIVISION', 'ORDER_TYPE', 'Inventory Status', 'PCKDTE',
        'SHIP_DATE_TIME', 'SHIP_STS', 'Sales_office', 'SF_Tracking_num'],
       dtype='object')]

***summary***

In [103]:
# line count
def line_count(df1, df2, date1, date2, line1, line2):
    line_df = pd.DataFrame(df1.groupby(
        date1)[line1].nunique()
        ).reset_index().merge(
            pd.DataFrame(
            df2.groupby(date2)[line2].nunique()
            ).reset_index(),
    left_on = date1, right_on = date2, how = 'outer')
    line_df = line_df.drop(date2, axis = 1)
    line_df.columns = ['date_time', 'bdp_line_ct','ou_line_ct']
    line_df=line_df.dropna(subset = ['ou_line_ct'])
    return line_df

In [104]:
line_df_out_bose = line_count(outb_0601_bose,bos_jun_ou_ou, 
    'closed_date', 'SHIP_DATE_TIME', 
    'order_line','CUSTOMER PO',)

In [105]:
bose_214_out1, bose_214_out2 = compare_data(
    outb_0601_bose,bos_jun_ou_ou, 
    'closed_date', 'SHIP_DATE_TIME', 
    'order_id','SHIPMENT_ID',
    'shipped_qty','TOTAL\nQTY'
)

In [106]:
bose_in_summary = bose_214_out1.dropna(
    subset =['bdp_hdr_ct']
    ).merge(
        line_df_out_bose, on = 'date_time'
        );bose_in_summary

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty,bdp_line_ct,ou_line_ct
0,20210601,138.0,138,3099.0,3099.0,454.0,76
1,20210602,97.0,97,11882.0,11882.0,179.0,69
2,20210604,333.0,333,33102.0,33102.0,615.0,159
3,20210606,14.0,14,29873.0,29873.0,14.0,6
4,20210607,126.0,126,25505.0,25505.0,292.0,82
5,20210608,234.0,234,31804.0,31822.0,461.0,107
6,20210609,176.0,176,30373.0,30373.0,302.0,107
7,20210610,171.0,171,11911.0,11911.0,288.0,101
8,20210611,196.0,196,45614.0,45614.0,356.0,113
9,20210613,19.0,19,39905.0,39905.0,36.0,15


> those data has difference. 

In [107]:
# data has difference.
bose_214_out2

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
5,20210608,234.0,234,31804.0,31822.0
12,20210616,130.0,130,17912.0,18754.0
13,20210617,193.0,193,9652.0,9932.0
15,20210620,18.0,18,40348.0,41041.0
20,20210625,188.0,188,31864.0,32620.0
24,20210701,276.0,216,21252.0,20019.0


detail_out check 

In [207]:
def detail_check(inb_bose_bdp, inb_bose_oud, wrong_time, cols_bdp, cols_ou):
    """
    输入两个表名字后, 写有问题的时间查看细节区别,
    cols = id, qty, date
    """
    df_detail = pd.DataFrame(
        inb_bose_bdp[inb_bose_bdp[cols_bdp[2]] == wrong_time][
            cols_bdp
            ].groupby(cols_bdp[0])[cols_bdp[1]].sum()
            ).reset_index().merge(
                pd.DataFrame(inb_bose_oud[inb_bose_oud[cols_ou[2]] == wrong_time][
                    cols_ou
                    ].groupby(cols_ou[0])[cols_ou[1]].sum()).reset_index(),
    left_on = cols_bdp[0], right_on = cols_ou[0], how = 'outer'
)   
    df_detail.columns = ['bdp_id','bdp_qty','ou_id','ou_qty']
    df_detail2 = df_detail[df_detail['bdp_qty'] - df_detail['ou_qty'] != 0]

    return df_detail,df_detail2


In [481]:
# outb_0601_bose
# bos_jun_ou_ou
    # 'closed_date', 'SHIP_DATE_TIME', 
    # 'order_id','SHIPMENT_ID',
    # 'shipped_qty','TOTAL\nQTY'

bose_detail1, bose_detail2 = detail_check(outb_0601_bose, bos_jun_ou_ou, wrong_time='20210608',
    cols_bdp = ['order_id', 'shipped_qty', 'closed_date'],
    cols_ou = ['SHIPMENT_ID','TOTAL\nQTY','SHIP_DATE_TIME'])

In [482]:
bose_detail1

,bdp_id,bdp_qty,ou_id,ou_qty
0,0324502845,1.00,0324502845,1.00
1,0324502846,1.00,0324502846,1.00
2,0324502871,1.00,0324502871,1.00
3,0324503134,1.00,0324503134,1.00
4,0324503135,1.00,0324503135,1.00
...,...,...,...,...
229,89935715,450.00,89935715,450.00
230,89935716,"23,956.00",89935716,"23,956.00"
231,89935775,12.00,89935775,12.00
232,89935776,4.00,89935776,4.00


> '20210608' detail diffs

In [483]:
bose_detail2

,bdp_id,bdp_qty,ou_id,ou_qty
221,89935066,18.00,89935066,36.00


In [484]:
# outb_0601_bose_new = outb_0601_bose[outb_0601_bose['closed_date'] == '20210616'][['order_id','shipped_qty']]
# bos_jun_ou_ou_new = bos_jun_ou_ou[bos_jun_ou_ou['SHIP_DATE_TIME'] == '20210616'][['SHIPMENT_ID','TOTAL\nQTY']]
# bos_jun_ou_ou_new.columns = outb_0601_bose_new.columns


# [outb_0601_bose_new['shipped_qty'].sum(),
# bos_jun_ou_ou_new['shipped_qty'].sum()]

_____

<span style="font-size: 12px"><i>Jupyter Notebook **Scale_Bose_CN214.ipynb**:<br>
**Siming Yan** created on July 28, 2021<br>
</i></span>

In [281]:
# bos_jun_ou_ou[bos_jun_ou_ou['SHIPMENT_ID'] == '89937858']
# outb_0601_bose[outb_0601_bose['order_id'] == '89937858']

---
<span style="color:rgba(255,244,0,1);font-size:42px"><b>MICHELIN</b></span> 


### inb

In [45]:
 
## bdp dwd final table.
mich_in_bdp = pd.read_csv('./CN-267/inbline_miche_bdp.csv')
# mich_in_bdp = pd.read_csv('./CN-267/mich_in_bdp_mid.csv')
db_mich_in = mich_in_bdp 

In [46]:
## 业务数据
# bos_jun_in_ou = pd.read_excel("./Bose June Inbound.xls")
mich_in_ou = pd.read_excel(
    "./CN-267/CN-267-inbound.xlsx", header = 0, sheet_name = 'Sheet1',
    index_col = None);mich_in_ou.head(2)
# mich_in_ou = mich_in_ou.dropna(subset = ['SHIP_DATE_TIME'])

# mich_in_bdp2 = mich_in_bdp
mich_in_ou2 = mich_in_ou


In [47]:
# # db_mich_in = pd.read_clipboard()
# names  = ['receipt_id', 'warehouse', 
# 'INTERNAL_RECEIPT_NUM', 'ITEM', 'ITEM_CATEGORY5', 
# 'ORIGINAL_TOTAL_QUANTITY', 'TOTAL_QTY', 
# 'ERP_ORDER_LINE_NUM', 'CLOSE_DATE', 'CREATION_DATE_TIME_STAMP']
# db_mich_in = db_mich_in[names]
# db_mich_in = db_mich_in.rename({'receipt_id':'RECEIPT_ID'}, axis = 1)
# db_mich_in.to_csv('./db_mich_in.csv', index = False)
# # mich_in_ou_01 = mich_in_ou[mich_in_ou['CLOSE_DATE'] == '20210601']
db_mich_in.columns

Index(['wms_company_id', 'sku_code', 'asn_id', 'original_qty', 'receive_qty',
       'create_time', 'closed_date', 'asn_line_id', 'inc_day'],
      dtype='object')

In [48]:
date_pre(mich_in_ou, ['CLOSE_DATE'])
date_pre(db_mich_in, ['closed_date'])

,wms_company_id,sku_code,asn_id,original_qty,receive_qty,create_time,closed_date,asn_line_id,inc_day
0,Michelin,538717_126,3830931_20,1.0,1.0,2021-05-31 02:06:00,20210601,974332.0,20210601


In [49]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=30)
# print(db_mich_in.columns.to_numpy())

# pd.set_option("display.max_rows", None, "display.max_columns", None)
# pd.options.display.max_rows = 31
# pd.options.display.max_columns = 12

# pd.set_option("display.max_rows = 31", None, "display.max_columns = 15", None)
# pd.DataFrame(db_mich_in.groupby(['RECEIPT_ID', 'ITEM_CATEGORY5'])['TOTAL_QTY'].sum()).reset_index().merge(
#     mich_in_ou_01.groupby('RECEIPT_ID')['TOTAL_QTY'].sum(),
#     on = 'RECEIPT_ID',
#     how = 'outer'
# )


# # ----  it seems that we need to filter out receipt Id  start with LSC .

### 筛选条件.

In [50]:
db_mich_in2 = db_mich_in[db_mich_in['asn_id'].str.slice(0,3) != 'LSC'] 
db_mich_in2 = db_mich_in2[db_mich_in['asn_id'].str.slice(0,4) != 'SISC'] 
db_mich_in2 = db_mich_in2[db_mich_in['asn_id'].str.slice(0,3) != 'CPP']
# db_mich_in2 = db_mich_in2[db_mich_in['asn_id'].str.slice(0,4) != 'CTSC']

<ipython-input-50-408fcbef9491>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  db_mich_in2 = db_mich_in2[db_mich_in['asn_id'].str.slice(0,4) != 'SISC']
<ipython-input-50-408fcbef9491>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  db_mich_in2 = db_mich_in2[db_mich_in['asn_id'].str.slice(0,3) != 'CPP']


In [51]:
mich_in_df1, mich_in_df2 = compare_data(db_mich_in2, mich_in_ou,
'closed_date','CLOSE_DATE',
'asn_id','RECEIPT_ID',
col2_quant = 'TOTAL_QTY', col1_quant = 'receive_qty'
)



In [52]:
mich_in_df1

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,23,23,10954.0,10954
1,20210602,17,17,6490.0,6490
2,20210603,25,25,10739.0,10739
3,20210604,17,17,6957.0,6957
4,20210605,18,18,7307.0,7307
5,20210606,20,20,7541.0,7541
6,20210607,22,22,9039.0,9039
7,20210608,29,28,11747.0,11680
8,20210609,15,15,5280.0,5280
9,20210610,16,16,5655.0,5655


In [53]:
mich_in_df2

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
7,20210608,29,28,11747.0,11680


In [54]:
np.setdiff1d(db_mich_in2[db_mich_in2['closed_date'] == '20210608']['asn_id'], 
mich_in_ou[mich_in_ou['CLOSE_DATE'] == '20210608']['RECEIPT_ID'])

# CPP21062602, SISC21060501,
# CTSC21060801,
# CPP21062601, CPP3960847, CPP396084701 

array(['CTSC21060801'], dtype=object)

In [55]:
mich_in_ou[mich_in_ou['RECEIPT_ID'] == 'CTSC21060801']

,CLOSE_DATE,RECEIPT_ID,RECEIPT_ID_TYPE,RECEIPT_TYPE,Trailing_Status,Leading_Status,SOURCE_ID,ITEM,ERP_ORDER_LINE_NUM,SKU_MARKET,...,TIRE_TYPE,TIRE_TYPE_DESC,LPR_CATEGORY,ITEM_DIM,MANUALLY_ENTERED,RD_MARKET,CREATION_DATE_TIME_STAMP,ITEM_DESC,UNIT_WEIGHT,试验编码


In [300]:
# line_count 
def line_count(df1, df2, date1, date2, line1, line2):
    line_df = pd.DataFrame(df1.groupby(
        date1)[line1].nunique()
        ).reset_index().merge(
            pd.DataFrame(
            df2.groupby(date2)[line2].nunique()
            ).reset_index(),
    left_on = date1, right_on = date2, how = 'outer')
    line_df = line_df.drop(date2, axis = 1)
    line_df.columns = ['date_time', 'bdp_line_ct','ou_line_ct']
    line_df=line_df.dropna(subset = ['ou_line_ct'])
    line_df['diff'] = line_df['bdp_line_ct'] - line_df['ou_line_ct'] 
    line_df2 = line_df[line_df['diff'] != 0]
    return line_df, line_df2

In [301]:
db_mich_in.columns

Index(['wms_company_id', 'sku_code', 'asn_id', 'original_qty', 'receive_qty',
       'create_time', 'closed_date', 'asn_line_id', 'inc_day'],
      dtype='object')

In [302]:
mich_in_ou.loc[:,'new_line'] = mich_in_ou[['RECEIPT_ID', 'ERP_ORDER_LINE_NUM']].astype(str).agg('-'.join, axis=1)
db_mich_in2.loc[:,'new_line'] = db_mich_in2[['asn_id', 'asn_line_id']].astype(str).agg('-'.join, axis=1)

In [305]:
line_count(df1 = db_mich_in2, df2 = mich_in_ou, date1 = 'closed_date',date2 = 'CLOSE_DATE', 
line1 = 'new_line', 
line2 = 'new_line')[0]

,date_time,bdp_line_ct,ou_line_ct,diff
0,20210601,388,388,0
1,20210602,398,398,0
2,20210603,364,364,0
3,20210604,366,366,0
4,20210605,373,373,0
5,20210606,375,375,0
6,20210607,351,351,0
7,20210608,547,546,1
8,20210609,344,344,0
9,20210610,390,390,0


In [306]:
line_count(df1 = db_mich_in2, df2 = mich_in_ou, date1 = 'closed_date',date2 = 'CLOSE_DATE', 
line1 = 'new_line', 
line2 = 'new_line')[1]

,date_time,bdp_line_ct,ou_line_ct,diff
7,20210608,547,546,1
24,20210625,111,109,2
25,20210626,231,220,11


In [563]:
# detail_check(db_mich_in2, mich_in_ou,  wrong_time = '20210606',
# cols_bdp = ['RECEIPT_ID', 'TOTAL_QTY', 'CLOSE_DATE'],
# cols_ou = ['RECEIPT_ID', 'TOTAL_QTY', 'CLOSE_DATE']
# )


In [434]:
# np.setdiff1d(mich_in_ou[mich_in_ou['CLOSE_DATE'] == '20210603']['RECEIPT_ID'].unique() , 
# db_mich_in[db_mich_in['CLOSE_DATE'] == '20210603']['RECEIPT_ID'].unique() )

array(['B210602045SWT', 'B210602048SWT', 'B210602053SWT'], dtype=object)

---

In [19]:
 
#  col modi
re1 = re.compile(r'(?<=\.).+')
mich_in_bdp.columns = [re1.findall(i)[0] for i in list(mich_in_bdp.columns.to_numpy())] 


mich_in_bdp['closed_date'] = pd.to_datetime(mich_in_bdp['closed_date'])
mich_in_bdp['closed_date'] = [i.strftime('%Y%m%d') for i in mich_in_bdp['closed_date']]

date_pre(mich_in_ou, ['CLOSE_DATE'])

,CLOSE_DATE,RECEIPT_ID,RECEIPT_ID_TYPE,RECEIPT_TYPE,Trailing_Status,Leading_Status,SOURCE_ID,ITEM,ERP_ORDER_LINE_NUM,SKU_MARKET,...,TIRE_TYPE,TIRE_TYPE_DESC,LPR_CATEGORY,ITEM_DIM,MANUALLY_ENTERED,RD_MARKET,CREATION_DATE_TIME_STAMP,ITEM_DESC,UNIT_WEIGHT,试验编码
0,20210603,ES_32359_21,IM,OE,Closed,Closed,620,439287_117,1,OE,...,TL,TL,T,20,NaN,OE,2021-04-10 09:11:01.867,255/35R20 97W XL TL PILOT SPORT 4 VOL MIAO VCC...,11.05,3.0498


---

### outb

In [628]:

## bdp dwd final table.
# mich_out_bdp = pd.read_excel('./CN-267/mich_inandout_06.xlsx', sheet_name = 'Sheet2')
# mich_out_bdp = pd.read_csv('./CN-267/join_bdp_mich_out.csv')
# mich_out_bdp = pd.read_csv('./CN-267/oub_mich_bdp.csv') # # worked

mich_out_bdp = pd.read_csv('./CN-267/mich_asdjk.csv')


C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [629]:

mich_out_bdp = colname_bdp(mich_out_bdp)
mich_out_bdp.head()
 

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,sku_code,sku_name,sku_desc,sku_barcode,batch_number,serial_number,...,std_shipped_weight,priority,closed_date,create_date,last_modified_date,src_inc_day,data_source,shipment_id,inc_day,src
0,Michelin,Michelin,AOE,AOE,665821_102,665821_102,255/50 R20 109V XL TL PRIMACY SUV ACOUSTIC DT1...,LTD,NaN,NaN,...,NaN,0,2021/6/11 12:26,2021/6/11 3:46,2021/6/11 4:26,20210803,scale_michelin,178602,20210611,scale
1,Michelin,Michelin,AOE,AOE,665821_102,665821_102,255/50 R20 109V XL TL PRIMACY SUV ACOUSTIC DT1...,LTD,NaN,NaN,...,NaN,0,2021/7/18 18:19,2021/7/15 4:36,2021/7/18 10:19,20210803,scale_michelin,186555,20210718,scale
2,Michelin,Michelin,AOE,AOE,665821_102,665821_102,255/50 R20 109V XL TL PRIMACY SUV ACOUSTIC DT1...,LTD,NaN,NaN,...,NaN,0,2021/7/18 18:19,2021/7/15 4:36,2021/7/18 10:19,20210803,scale_michelin,186558,20210718,scale
3,Michelin,Michelin,AGB,AGB,839180_102,839180_102,215/50 R18 92W TL PRIMACY 3 ST AO GRNX MIAO P7...,NaN,NaN,NaN,...,NaN,0,2021/6/1 14:52,2021/5/28 6:06,2021/6/1 6:52,20210803,scale_michelin,175381,20210601,scale
4,Michelin,Michelin,AGB,AGB,149663_102,149663_102,245/45 R18 100Y EXTRA LOAD TL PILOT SPORT 3 ST...,NaN,NaN,NaN,...,NaN,0,2021/6/1 14:52,2021/5/28 6:06,2021/6/1 6:52,20210803,scale_michelin,175381,20210601,scale


In [626]:
 
## bdp dwd final table.
# mich_out_bdp = pd.read_csv('./CN-267/miche_out_bdp.csv')

## 业务数据 
mich_out_ou = pd.read_excel("./CN-267/CN-267-outbound.xlsx", header = 0, sheet_name = 'Sheet1')
mich_out_ou.columns

Index(['类别', 'MSG_TYPE', 'SHIP_DATE_TIME', 'COMPANY', 'SHIP_TO',
       'SHIP_TO_NAME', 'CARRIER', 'SHIPMENT_ID', 'C', 'DO_LINE_NBR',
       'IR_SEQ_NBR', 'ITEM', 'REQUESTED_QTY', 'TOTAL\nQTY', 'REQUESTED_OWNER',
       'REQUESTED_SUBINV', 'ORDER_TYPE', 'SKU MARKET', 'PCKDTE',
       'CREATE_DATE_TIME', 'SHIP_STS', 'PRODUCTION_LINE',
       'PRODUCTION_LINE_DESC', 'BUSINESS_LINE', 'BUSINESS_LINE_DESC', 'BRAND',
       'BRAND_DESC', 'TIRE_TYPE', 'TIRE_TYPE_DESC', 'LPR_CATEGORY',
       'LPR_CATEGORY_DESC', 'SEASON', 'SEASON_DESC', 'Tire_R', 'Tire_W',
       'UNIT_WEIGHT', 'ROUTING_INS', 'SHIPPING_INS', '试验编码'],
      dtype='object')

In [619]:
mich_out_ou['SHIPMENT_ID'] = mich_out_ou['SHIPMENT_ID'].astype(str)
mich_out_bdp['shipment_id'] = mich_out_bdp['shipment_id'].astype(str)
mich_out_bdp.columns
 

Index(['wms_company_id', 'wms_company_name', 'wms_warehouse_id',
       'wms_warehouse_name', 'sku_code', 'sku_name', 'sku_desc', 'sku_barcode',
       'batch_number', 'serial_number', 'vendor_sku_barcode', 'order_id',
       'order_line', 'order_status', 'order_type', 'launch_number', 'pick_no',
       'pick_create_date', 'required_ship_date', 'inventory_status',
       'original_order_qty', 'allocated_qty', 'shipped_qty', 'shipped_volume',
       'shipped_weight', 'quantity_um', 'volume_um', 'weight_um',
       'std_shipped_volume', 'std_shipped_weight', 'priority', 'closed_date',
       'create_date', 'last_modified_date', 'src_inc_day', 'data_source',
       'shipment_id', 'inc_day', 'src'],
      dtype='object')

In [620]:
mich_out_ou2 = mich_out_ou[['SHIP_DATE_TIME', 'SHIPMENT_ID', 'TOTAL\nQTY', 'CREATE_DATE_TIME', 'DO_LINE_NBR']]

In [646]:
mich_out_bdp = mich_out_bdp[mich_out_bdp['wms_warehouse_id'] == 'AOE']

mich_out_bdp['closed_date'] = [datetime.strptime(
     i, '%Y/%m/%d %H:%M'
     ).strftime('%Y%m%d') for i in mich_out_bdp['closed_date'] ]
 

In [641]:
date_pre(mich_out_ou2, ['SHIP_DATE_TIME'])
date_pre(mich_out_bdp, ['closed_date'])

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,sku_code,sku_name,sku_desc,sku_barcode,batch_number,serial_number,...,std_shipped_weight,priority,closed_date,create_date,last_modified_date,src_inc_day,data_source,shipment_id,inc_day,src
0,Michelin,Michelin,AOE,AOE,665821_102,665821_102,255/50 R20 109V XL TL PRIMACY SUV ACOUSTIC DT1...,LTD,NaN,NaN,...,NaN,0,20210611,2021/6/11 3:46,2021/6/11 4:26,20210803,scale_michelin,178602,20210611,scale


In [642]:
?compare_data

Signature:
compare_data(
    df1,
    df2,
    date1,
    date2,
    col1_line,
    col2_line,
    col1_quant,
    col2_quant,
)
Docstring:
数据对比函数, 输出两个df, 一个为全部的对比值, 另一个为有差异的subset
两个df对比(先写bdp的数据), 两个时间戳, 两个line字段名称, 两个quant字段名称
File:      c:\users\dscshap3808\documents\data\cn-214\<ipython-input-264-576333283167>
Type:      function


In [647]:
mich_out1, mich_out2 = compare_data(
    mich_out_bdp, mich_out_ou2,
'closed_date', 'SHIP_DATE_TIME',
'order_id', 'SHIPMENT_ID',
col2_quant = 'TOTAL\nQTY', col1_quant = 'shipped_qty'
)

In [649]:
mich_out1.dropna()

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,2,2.0,1414.0,1414.0
1,20210602,6,7.0,1861.0,2051.0
2,20210603,6,6.0,4376.0,4376.0
3,20210604,4,6.0,3224.0,3960.0
4,20210605,11,11.0,9224.0,9224.0
5,20210606,4,4.0,3560.0,3560.0
6,20210607,5,5.0,4068.0,4068.0
7,20210608,12,14.0,7358.0,7458.0
8,20210609,14,14.0,7189.0,7189.0
9,20210610,9,9.0,4341.0,4341.0


In [650]:
mich_out2

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
1,20210602,6,7.0,1861.0,2051.0
3,20210604,4,6.0,3224.0,3960.0
7,20210608,12,14.0,7358.0,7458.0
10,20210611,16,17.0,11003.0,11107.0
12,20210613,9,10.0,7542.0,7811.0
15,20210616,10,11.0,6583.0,8008.0
16,20210617,14,17.0,8305.0,8369.0
21,20210622,16,17.0,9040.0,9190.0
23,20210624,10,13.0,6395.0,9326.0
24,20210625,10,11.0,6955.0,8455.0


In [656]:
np.setdiff1d(mich_out_ou2[mich_out_ou2['SHIP_DATE_TIME'] == '20210608']['SHIPMENT_ID'], 
mich_out_bdp[mich_out_bdp['closed_date'] == '20210608']['order_id'].astype(str))

# SISC21060501,CTSC21060801,CPP21062602, CPP21062601, CPP3960847, CPP396084701 

array(['CTSC21060801', 'CTSC21060802'], dtype=object)

In [653]:
np.setdiff1d(mich_out_bdp[mich_out_bdp['closed_date'] == '20210606']['order_id'].astype(str), 
mich_out_ou2[mich_out_ou2['SHIP_DATE_TIME'] == '20210606']['SHIPMENT_ID'])

# SISC21060501,CTSC21060801,CPP21062602, CPP21062601, CPP3960847, CPP396084701 

array([], dtype=object)

In [660]:
def mich(time1):
    return np.setdiff1d(mich_out_ou2[mich_out_ou2['SHIP_DATE_TIME'] == time1]['SHIPMENT_ID'], 
    mich_out_bdp[mich_out_bdp['closed_date'] == time1]['order_id'].astype(str))
    

tab = []
for i in mich_out2['date_time']:
    tab.append(mich(i))

In [661]:
tab

[array(['CTSC21060201'], dtype=object),
 array(['CTSC21060401', 'SITC21060401'], dtype=object),
 array(['CTSC21060801', 'CTSC21060802'], dtype=object),
 array(['CTSC21061001'], dtype=object),
 array(['CTSC21061301'], dtype=object),
 array(['CTSC21061601'], dtype=object),
 array(['CTSC21061701', 'CTSC21061702', 'CTSC21061703'], dtype=object),
 array(['CTSC21062201'], dtype=object),
 array(['CTSC21062301', 'CTSC21062302', 'SITC21062401'], dtype=object),
 array(['CTSC21062501'], dtype=object),
 array(['CTSC21062601'], dtype=object),
 array(['CTSC21062901'], dtype=object)]

In [512]:
pd.DataFrame(
    mich_out_bdp.groupby(['closed_date', 'order_id'])['order_line'].count()
    ).reset_index().merge(
        pd.DataFrame(
            mich_out_ou2.groupby(['SHIP_DATE_TIME', 'SHIPMENT_ID'])['DO_LINE_NBR'].count()
    ).reset_index(),
    left_on = ['closed_date', 'order_id'], right_on = ['SHIP_DATE_TIME', 'SHIPMENT_ID']
    )

,closed_date,order_id,order_line,SHIP_DATE_TIME,SHIPMENT_ID,DO_LINE_NBR
0,20210601,3874898,1,20210601,3874898,1
1,20210601,5400001558,1,20210601,5400001558,1
2,20210602,3854168,1,20210602,3854168,1
3,20210602,3875116,2,20210602,3875116,2
4,20210602,3875440,1,20210602,3875440,1
...,...,...,...,...,...,...
286,20210630,3962937,1,20210630,3962937,1
287,20210630,3962954,1,20210630,3962954,1
288,20210630,3976997,1,20210630,3976997,1
289,20210630,3981428,1,20210630,3981428,1


---

<span style="color:rgba(255,244,0,1);font-size:42px"><b>Fas-054</b></span> 

> inbound

In [5]:
os.getcwd()
# import ciso8601

'c:\\Users\\dscshap3808\\Documents\\data\\CN-214'

In [6]:
pernod_057_in_oud = pd.read_csv('./CN-057_PERNOD/pernod_057_in_oud.csv')

pernod_in_bdp = pd.read_csv('./CN-057_PERNOD/fas_inb_bdp.csv')


In [7]:

# re1 = re.compile(r'(?<=\.).+')
# pernod_in_bdp.columns = [re1.findall(i)[0] for i in list(pernod_in_bdp.columns.to_numpy())]

In [8]:
from datetime import datetime
pernod_in_bdp['closed_date'] = [
    datetime.strptime(i, '%Y/%m/%d %H:%M') for i in pernod_in_bdp['closed_date']]

pernod_in_bdp['closed_date'] =  pernod_in_bdp['closed_date'] - pd.Timedelta(hours=8) 
# bdp - 8 h so that the data can be matched;.

In [9]:
[pernod_057_in_oud.columns, pernod_in_bdp.columns]

[Index(['Unnamed: 0', 'WH', 'COMPANY', 'Create DT', 'Confirm DT', 'Product',
        'Description', 'TYPE', 'Status', 'TRSC #', 'Ship ID', 'INV', 'PO',
        'INB#', 'contract_no', 'From Loc', 'To Loc', 'Qty', 'UOM', 'BottlingDT',
        'Transaction_type', 'ALLOCATION_ZONE'],
       dtype='object'),
 Index(['wms_company_id', 'sku_code', 'asn_id', 'original_qty', 'receive_qty',
        'create_time', 'closed_date', 'asn_line_id', 'inc_day'],
       dtype='object')]

In [10]:
pernod_057_in_oud['closed_date'] = [datetime.strptime(
     i, '%m/%d/%Y'
     ).strftime('%Y%m%d') for i in pernod_057_in_oud['Confirm DT'] ]
date_pre(pernod_057_in_oud, ['Create DT'])
date_pre(pernod_in_bdp, ['closed_date'])

,wms_company_id,sku_code,asn_id,original_qty,receive_qty,create_time,closed_date,asn_line_id,inc_day
0,Pernod,30087,ONEYGOTB03746500,13860,13860,2021/5/28 8:37,20210601,248949,20210601


In [11]:
?groupby

Object `groupby` not found.


In [12]:
pernod_057_in_oud2 = pernod_057_in_oud.groupby('TRSC #', as_index = False).first()

In [13]:
linwuqi1, linwuqi2 = compare_data(
    pernod_in_bdp, pernod_057_in_oud2,
'closed_date', 'closed_date',
'asn_id', 'Ship ID',
'receive_qty', 'Qty'
)

In [14]:
linwuqi1

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,9,9,166119,166119
1,20210602,17,17,249720,249720
2,20210603,10,10,160032,160032
3,20210604,6,6,66327,66327
4,20210607,11,11,125235,125235
5,20210608,13,13,203460,203460
6,20210609,6,6,88830,88830
7,20210610,4,4,59040,59040
8,20210611,4,4,71785,71785
9,20210615,5,5,122760,122760


In [15]:
linwuqi1['ou_hdr_ct'].sum()
# 196 
# 3596041

196

In [16]:
linwuqi2

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty


In [24]:
?detail_check

Object `detail_check` not found.


In [17]:
np.setdiff1d(
    pernod_057_in_oud2[pernod_057_in_oud2['closed_date'] == '20210608']['Ship ID'].unique(),
    pernod_in_bdp[pernod_in_bdp['closed_date'] == '20210608']['asn_id'].unique()
    
)


array([], dtype=object)

In [74]:
detail_check(
    pernod_in_bdp, pernod_057_in_oud2, '20210608',
    ['asn_id',  'receive_qty', 'closed_date'],
    ['Ship ID', 'Qty', 'closed_date' ])[0]

NameError: name 'pernod_in_bdp' is not defined

In [19]:
[pernod_057_in_oud2.columns, pernod_in_bdp.columns]

[Index(['TRSC #', 'Unnamed: 0', 'WH', 'COMPANY', 'Create DT', 'Confirm DT',
        'Product', 'Description', 'TYPE', 'Status', 'Ship ID', 'INV', 'PO',
        'INB#', 'contract_no', 'From Loc', 'To Loc', 'Qty', 'UOM', 'BottlingDT',
        'Transaction_type', 'ALLOCATION_ZONE', 'closed_date'],
       dtype='object'),
 Index(['wms_company_id', 'sku_code', 'asn_id', 'original_qty', 'receive_qty',
        'create_time', 'closed_date', 'asn_line_id', 'inc_day'],
       dtype='object')]

In [31]:
pd.DataFrame(
    pernod_in_bdp.groupby(['closed_date', 'asn_id'])['asn_line_id'].count()
    ).reset_index().merge(
        pd.DataFrame(
            pernod_057_in_oud2.groupby(['closed_date', 'Ship ID'])['TRSC #'].count()
    ).reset_index(),
    left_on = ['closed_date', 'asn_id'], right_on = ['closed_date', 'Ship ID']
    )

,closed_date,asn_id,asn_line_id,Ship ID,TRSC #
0,20210601,ONEYGOTB03746500,1,ONEYGOTB03746500,12
1,20210601,ONEYLIVA68562800,2,ONEYLIVA68562800,2
2,20210601,ONEYLIVA68572900,1,ONEYLIVA68572900,1
3,20210601,ONEYLIVB11758400,2,ONEYLIVB11758400,2
4,20210601,ONEYLIVB11764800,4,ONEYLIVB11764800,9
...,...,...,...,...,...
191,20210627,ONEYLIVB20547700,2,ONEYLIVB20547700,2
192,20210627,ONEYLIVB20557800,3,ONEYLIVB20557800,3
193,20210627,ONEYLIVB20569400,1,ONEYLIVB20569400,21
194,20210627,OOLU2121146170,1,OOLU2121146170,1


In [32]:
[pernod_057_in_oud2[pernod_057_in_oud2['Ship ID'] == 'ONEYGOTB03746500']['Qty'],pernod_057_in_oud2[pernod_057_in_oud2['Ship ID'] == 'ONEYGOTB03746500']['Qty'].sum(),
pernod_in_bdp[pernod_in_bdp['asn_id'] == 'ONEYGOTB03746500']]

[19    1056
 20    1056
 21    1056
 22    1188
 23    1188
 24    1188
 25    1188
 26    1188
 27    1188
 28    1188
 29    1188
 30    1188
 Name: Qty, dtype: int64,
 13860,
   wms_company_id sku_code            asn_id  original_qty  receive_qty  \
 0         Pernod    30087  ONEYGOTB03746500         13860        13860   
 
       create_time closed_date  asn_line_id   inc_day  
 0  2021/5/28 8:37    20210601       248949  20210601  ]

In [33]:
pd.concat([
    pernod_057_in_oud[pernod_057_in_oud['Ship ID'] == 'ONEYLIVB16831800'][['Ship ID', 'Qty']], 
    pernod_in_bdp[pernod_in_bdp['asn_id'] == 'ONEYLIVB16831800'][['asn_id', 'receive_qty']] 
    ],
    axis = 1)


,Ship ID,Qty,asn_id,receive_qty
0,ONEYLIVB16831800,600.0,NaN,NaN
1,ONEYLIVB16831800,6000.0,NaN,NaN
2,ONEYLIVB16831800,6000.0,NaN,NaN
3,ONEYLIVB16831800,600.0,NaN,NaN
4,ONEYLIVB16831800,6000.0,NaN,NaN
5,ONEYLIVB16831800,6000.0,NaN,NaN
6,ONEYLIVB16831800,6000.0,NaN,NaN
7,ONEYLIVB16831800,600.0,NaN,NaN
8,ONEYLIVB16831800,6000.0,NaN,NaN
21,NaN,NaN,ONEYLIVB16831800,12000.0


> outbound

In [20]:
pernod_057_out_oud = pd.read_csv('./CN-057_PERNOD/pernod_057_out_oud.csv')
pernod_out_bdp = pd.read_csv('./CN-057_PERNOD/fas_ofaosojo.csv')

In [21]:
re1 = re.compile(r'(?<=\.).+')
pernod_out_bdp.columns = [re1.findall(i)[0] for i in list(pernod_out_bdp.columns.to_numpy())]

In [22]:
pernod_out_bdp['closed_date'] = [
    datetime.strptime(i, '%Y-%m-%d %H:%M:%S') for i in pernod_out_bdp['closed_date']]

pernod_out_bdp['closed_date'] =  pernod_out_bdp['closed_date'] - pd.Timedelta(hours=8) 
# bdp - 8 h so that the data can be matched;.

In [23]:
pernod_057_out_oud['closed_date'] = [datetime.strptime(
     i, '%m/%d/%Y'
     ).strftime('%Y%m%d') for i in pernod_057_out_oud['Confirm DT'] ]
date_pre(pernod_057_out_oud, ['Create DT'])
date_pre(pernod_out_bdp, ['closed_date'])

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,sku_code,sku_name,sku_desc,sku_barcode,batch_number,serial_number,...,std_shipped_weight,priority,closed_date,create_date,last_modified_date,src_inc_day,data_source,shipment_id,inc_day,src
0,Pernod,Pernod,FAS,FAS,DBL0N7,DBL0N7,传奇源自拉菲罗斯柴尔德波尔多红葡萄酒,NaN,NaN,NaN,...,NaN,0,20210601,2021-05-28 09:32:02,2021-06-01 05:29:33,20210803,scale_fas,156167,20210601,scale


In [24]:
pernod_057_out_oud2 = pernod_057_out_oud.groupby('TRSC #').first()

In [25]:
pernod_057_out_oud2 = pernod_057_out_oud2[pernod_057_out_oud2['Ship ID'].str.fullmatch(r'\d+')]

In [26]:
[pernod_out_bdp.columns, pernod_057_out_oud2.columns]

[Index(['wms_company_id', 'wms_company_name', 'wms_warehouse_id',
        'wms_warehouse_name', 'sku_code', 'sku_name', 'sku_desc', 'sku_barcode',
        'batch_number', 'serial_number', 'vendor_sku_barcode', 'order_id',
        'order_line', 'order_status', 'order_type', 'launch_number', 'pick_no',
        'pick_create_date', 'required_ship_date', 'inventory_status',
        'original_order_qty', 'allocated_qty', 'shipped_qty', 'shipped_volume',
        'shipped_weight', 'quantity_um', 'volume_um', 'weight_um',
        'std_shipped_volume', 'std_shipped_weight', 'priority', 'closed_date',
        'create_date', 'last_modified_date', 'src_inc_day', 'data_source',
        'shipment_id', 'inc_day', 'src'],
       dtype='object'),
 Index(['Unnamed: 0', 'WH', 'COMPANY', 'Create DT', 'Confirm DT', 'Product',
        'Description', 'TYPE', 'Status', 'Ship ID', 'awb', 'INV', 'PO', 'INB#',
        'contract_no', 'From Loc', 'To Loc', 'Qty', 'UOM', 'BottlingDT',
        'Transaction_type', 'AL

In [27]:
linwuqi3, linwuqi4 = compare_data(
    pernod_out_bdp, pernod_057_out_oud2,
'closed_date', 'closed_date',
'order_id', 'Ship ID',
'shipped_qty', 'Qty'
)

In [28]:
[linwuqi3.dropna()['bdp_hdr_ct'].sum(), 
linwuqi3.dropna()['ou_hdr_ct'].sum(), 
linwuqi3.dropna()['bdp_qty'].sum(), 
linwuqi3.dropna()['ou_qty'].sum()


]

[1247, 1270.0, 1093417.0, 1094064.0]

In [42]:
linwuqi3

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,44,44.0,26780.0,26780.0
1,20210602,66,66.0,66431.0,66431.0
2,20210603,78,78.0,46201.0,46201.0
3,20210604,46,46.0,55460.0,55460.0
4,20210607,67,67.0,57734.0,57734.0
5,20210608,37,37.0,12330.0,12330.0
6,20210609,95,95.0,59746.0,59746.0
7,20210610,99,99.0,72240.0,72240.0
8,20210611,54,54.0,48008.0,48008.0
9,20210615,94,94.0,114314.0,114314.0


In [43]:
linwuqi4

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
12,20210618,79,102.0,84402.0,85049.0


In [31]:
def fas_line_check(time1):
    return np.setdiff1d(
    pernod_057_out_oud2[pernod_057_out_oud2['closed_date'] == time1]['Ship ID'].unique().astype(str),
    pernod_out_bdp[pernod_out_bdp['closed_date'] == time1]['order_id'].unique().astype(str)
)

tab_054=[]
for i in linwuqi4['date_time']:
    tab_054.append(fas_line_check(i))

# ['20210527 PRC-1']
# ['SHJN2105031 查验退曹路']
tab_054

[array(['31957240', '31957259', '31957261', '31957269', '31957314',
        '31957318', '31957321', '31957325', '31957326', '31957327',
        '31957328', '31957334', '31957336', '31957337', '31957348',
        '31957353', '31957355', '31957356', '31957357', '31957382',
        '31957407', '42022781', '42022782'], dtype='<U8')]

In [32]:
fas_line_check('20210618')

array(['31957240', '31957259', '31957261', '31957269', '31957314',
       '31957318', '31957321', '31957325', '31957326', '31957327',
       '31957328', '31957334', '31957336', '31957337', '31957348',
       '31957353', '31957355', '31957356', '31957357', '31957382',
       '31957407', '42022781', '42022782'], dtype='<U8')

In [41]:
# pernod_057_out_oud2[pernod_057_out_oud2['Ship ID'].isin(fas_line_check('20210618'))].to_csv(
#     './CN-057_PERNOD/diff_data.csv', encoding = 'utf_8_sig')

pernod_057_out_oud2[~pernod_057_out_oud2['Ship ID'].isin(fas_line_check('20210618'))]

,Unnamed: 0,WH,COMPANY,Create DT,Confirm DT,Product,Description,TYPE,Status,Ship ID,...,INB#,contract_no,From Loc,To Loc,Qty,UOM,BottlingDT,Transaction_type,ALLOCATION_ZONE,closed_date
TRSC #,,,,,,,,,,,,,,,,,,,,,
6426702,3,FAS,Pernod,5/28/2021,6/1/2021,DBL0N7,传奇源自拉菲罗斯柴尔德波尔多红葡萄酒 6 X 75CL,Order Pick,Jin2 Hold,42021943,...,NaN,None,4-14-C-24,SHIP-1,120,BT,11/25/2020 12:00:00 AM,Order Pick,A-FAS,20210601
6426706,1,FAS,Pernod,5/28/2021,6/1/2021,DBL119,传奇源自拉菲罗斯柴尔德波尔多白葡萄酒 6 X 75CL,Order Pick,Jin2 Normal,42021942,...,NaN,None,3-12-A-30,SHIP-1,60,BT,11/23/2020 12:00:00 AM,Order Pick,A-FAS,20210601
6426708,2,FAS,Pernod,5/28/2021,6/1/2021,DBL0N7,传奇源自拉菲罗斯柴尔德波尔多红葡萄酒 6 X 75CL,Order Pick,Jin2 Hold,42021942,...,NaN,None,4-14-C-24,SHIP-1,300,BT,11/25/2020 12:00:00 AM,Order Pick,A-FAS,20210601
6426712,0,FAS,Pernod,5/28/2021,6/1/2021,DBL0N7,传奇源自拉菲罗斯柴尔德波尔多红葡萄酒 6 X 75CL,Order Pick,Jin2 Hold,42021941,...,NaN,None,4-14-C-24,SHIP-1,120,BT,11/25/2020 12:00:00 AM,Order Pick,A-FAS,20210601
6426716,13,FAS,Pernod,5/28/2021,6/1/2021,DBC017,传奇源自拉菲罗斯柴尔德经典海星红葡萄酒 6 X 75CL,Order Pick,Jin2 Normal,42021940,...,NaN,None,3-11-A-16,SHIP-1,12,BT,10/26/2020 12:00:00 AM,Order Pick,A-FAS,20210601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6493234,4477,FAS,Pernod,6/28/2021,6/28/2021,DBL717,传奇源自拉菲罗斯柴尔德波尔多红葡萄酒 48 X 18.7CL,Order Pick,Paid Normal,31957475,...,NaN,None,4-18-D-13,SHIP-1,2,BT,12/31/4712 12:00:00 AM,Order Pick,A-FAS,20210628
6493236,4481,FAS,Pernod,6/28/2021,6/28/2021,BF2917,必富达粉红金酒风味配制酒 6 X 70CL,Order Pick,Jin2 Normal,31957475,...,NaN,None,2-32-A-09,SHIP-1,1,BT,12/12/2019 12:00:00 AM,Order Pick,A-FAS,20210628
6493240,4482,FAS,Pernod,6/28/2021,6/28/2021,DBS416,传说源自拉菲罗斯柴尔德圣爱美乐红葡萄酒 6 X 75CL,Order Pick,Paid Normal,21900346,...,NaN,None,FS-63-B-007,SHIP-1,1,BT,2/20/2018 12:00:00 AM,Order Pick,A-FAS,20210628


In [47]:
np.setdiff1d(
    pernod_out_bdp[pernod_out_bdp['closed_date'] == '20210608']['order_id'].unique().astype(str),
    pernod_057_out_oud2[pernod_057_out_oud2['closed_date'] == '20210608']['Ship ID'].unique()
)

array([], dtype='<U21')

<span style="color:rgba(255,244,0,1);font-size:42px"><b>pernod jiq-074</b></span> 




In [13]:
# jiq_inbound = pd.read_clipboard()
# jiq_inbound.to_csv('./CN-074_PERNOD/jiq_outb.csv', index = False, encoding = 'utf_8_sig')

In [14]:
jiq_inbound_ou = pd.read_csv('./CN-074_PERNOD/jiq_in.csv')
jiq_outbound_ou = pd.read_csv('./CN-074_PERNOD/jiq_outb.csv')
jiq_inb_bdp =  pd.read_csv('./CN-074_PERNOD/jiq_bdp_out.csv')
jiq_oub_bdp =  pd.read_csv('./CN-074_PERNOD/jiq_bdp_in.csv')

In [15]:
jiq_inbound_ou

,日期,外部物料号,物料名称,已收数量
0,2021/6/2,009527,马爹利尚选XXO干邑白兰地-有盒装,18
1,2021/6/2,015887,格兰威特单一麦芽苏格兰威士忌50年陈酿温切斯特典藏系列,2
2,2021/6/2,015868,格兰威特单一麦芽苏格兰威士忌50年陈酿温切斯特典藏系列,1
3,2021/6/2,015877,格兰威特单一麦芽苏格兰威士忌50年陈酿温切斯特典藏系列,1
4,2021/6/2,004687,马爹利XO干邑白兰地-有盒装,12
...,...,...,...,...
186,2021-06-17,DBL0N7,传奇源自拉菲罗斯柴尔德波尔多红葡萄酒 6 X 75CL,48402
187,2021-06-17,DBC017,传奇源自拉菲罗斯柴尔德经典海星红葡萄酒 6 X 75CL,8580
188,2021-06-17,DBR0N7,拉菲罗斯柴尔德珍藏波尔多法定产区红葡萄酒 6 X 75CL,3708
189,2021-06-29,DBS416,传说源自拉菲罗斯柴尔德圣爱美乐红葡萄酒 6 X 75CL,6


In [16]:
jiq_inbound_ou_1 = jiq_inbound_ou[ jiq_inbound_ou['日期'].str.match(r'\d+\/\d.+')]
jiq_inbound_ou_2 = jiq_inbound_ou[~jiq_inbound_ou['日期'].str.match(r'\d+\/\d.+')]

In [17]:
jiq_inbound_ou_1['日期'] = [
    datetime.strptime(i, '%Y/%m/%d') for i in jiq_inbound_ou_1['日期']]
jiq_inbound_ou_2['日期'] = [
    datetime.strptime(i, '%Y-%m-%d') for i in jiq_inbound_ou_2['日期']]

# pernod_in_bdp['closed_date'] =  pernod_in_bdp['closed_date'] - pd.Timedelta(hours=8) 
# bdp - 8 h so that the data can be matched;.

<ipython-input-17-f6cc7189119c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jiq_inbound_ou_1['日期'] = [
<ipython-input-17-f6cc7189119c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jiq_inbound_ou_2['日期'] = [


In [20]:
date_pre(jiq_inbound_ou_1, ['日期'])
date_pre(jiq_inbound_ou_2, ['日期'])

colname_bdp(jiq_inb_bdp)
date_pre(jiq_inb_bdp, ['create_date'])


,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,sku_code,sku_name,sku_desc,sku_barcode,batch_number,serial_number,...,std_shipped_weight,priority,closed_date,create_date,last_modified_date,src_inc_day,data_source,shipment_id,inc_day,src
0,Pernod-Wine,Pernod-Wine,JIQ,JIQ,035906,035906,绝对风味伏特加(覆盆莓味),NaN,NaN,NaN,...,NaN,0,2021-06-01 16:22:46,20210528,2021-06-01 08:23:09,20210803,scale_jiq,83294,20210601,scale


In [21]:
jiq_inbound_ou0 = pd.concat([jiq_inbound_ou_1, jiq_inbound_ou_2], axis = 0)

In [22]:
?compare_data

Signature:
compare_data(
    df1,
    df2,
    date1,
    date2,
    col1_line,
    col2_line,
    col1_quant,
    col2_quant,
)
Docstring:
数据对比函数, 输出两个df, 一个为全部的对比值, 另一个为有差异的subset
两个df对比(先写bdp的数据), 两个时间戳, 两个line字段名称, 两个quant字段名称
File:      c:\users\dscshap3808\documents\data\cn-214\<ipython-input-12-8e56aadf13ab>
Type:      function


In [23]:
jiq_inb_bdp.columns

Index(['wms_company_id', 'wms_company_name', 'wms_warehouse_id',
       'wms_warehouse_name', 'sku_code', 'sku_name', 'sku_desc', 'sku_barcode',
       'batch_number', 'serial_number', 'vendor_sku_barcode', 'order_id',
       'order_line', 'order_status', 'order_type', 'launch_number', 'pick_no',
       'pick_create_date', 'required_ship_date', 'inventory_status',
       'original_order_qty', 'allocated_qty', 'shipped_qty', 'shipped_volume',
       'shipped_weight', 'quantity_um', 'volume_um', 'weight_um',
       'std_shipped_volume', 'std_shipped_weight', 'priority', 'closed_date',
       'create_date', 'last_modified_date', 'src_inc_day', 'data_source',
       'shipment_id', 'inc_day', 'src'],
      dtype='object')

In [24]:
compare_data(
    jiq_inb_bdp, jiq_inbound_ou0,
    'create_date', '日期',
    col1_line= 'sku_code', col2_line = '外部物料号',
    col1_quant = 'shipped_qty',col2_quant = '已收数量'
)[0]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210526,5.0,NaN,256.0,NaN
1,20210527,1.0,NaN,50.0,NaN
2,20210528,1.0,NaN,120.0,NaN
3,20210531,1.0,NaN,480.0,NaN
4,20210601,15.0,2.0,1496.0,11004.0
5,20210602,13.0,10.0,1904.0,706.0
6,20210603,5.0,5.0,4194.0,422.0
7,20210604,3.0,NaN,2520.0,NaN
8,20210605,1.0,2.0,12.0,12099.0
9,20210607,10.0,2.0,5256.0,14.0


In [ ]:
checkl_detial()

<span style="color:rgba(255,244,0,1);font-size:42px"><b>FERRERO CN-274</b></span> 




In [111]:
os.getcwd()

'c:\\Users\\dscshap3808\\Documents\\data\\CN-214'

In [142]:
fer_inbound_ou = pd.read_excel('./CN-274_ferrero/CN-274 FERRERO CTU BTS WHS.xlsx', sheet_name = 4 )
fer_outbound_ou = pd.read_excel('./CN-274_ferrero/CN-274 FERRERO CTU BTS WHS.xlsx', sheet_name=  3 )
fer_inbound_bdp =  pd.read_csv('./CN-274_ferrero/fe_bdp_inb.csv') 
fer_outbound_bdp =  pd.read_csv('./CN-274_ferrero/fe_bdp_oub.csv')  

In [143]:
?colname_bdp

Signature: colname_bdp(df)
Docstring: <no docstring>
File:      c:\users\dscshap3808\documents\data\cn-214\<ipython-input-128-8060e98b97df>
Type:      function


In [146]:
colname_bdp(fer_inbound_bdp)
colname_bdp(fer_outbound_bdp)
date_pre(fer_inbound_ou, ['Date'])
date_pre(fer_inbound_bdp, ['closed_date'])
date_pre(fer_outbound_bdp, ['closed_date'])


In [169]:
date_pre(fer_outbound_ou, ['Post Date'])

,Seq #,Post Date,Created Date,Recv. Date,Order No,Ship To Name,Location,Product Code,Description,Lot,Ship Qty,STR Days,Volume,Unnamed: 13,Unnamed: 14
0,1.0,20210618,2021-06-16 14:00:03.263,20210606,1.084917e+09,NSO-成都,G023003,76867709.0,拉斐尔脆雪柔椰蓉扁桃仁糖果酥球原糖285粒装,L039AB154R,47.0,12.0,1.090165,NaN,NaN


In [152]:
[fer_inbound_ou.columns, fer_inbound_bdp.columns]

[Index(['Seq #', 'Date', 'Receipt ID', 'Warehouse', 'Location', 'Item',
        'Item Description', 'Lot', 'UOM', 'Putaway Qty', 'Volume'],
       dtype='object'),
 Index(['wms_company_id', 'wms_company_name', 'wms_warehouse_id',
        'wms_warehouse_name', 'internal_id', 'asn_id', 'asn_line_id',
        'asn_status', 'sku_code', 'sku_name', 'sku_desc', 'sku_barcode',
        'batch_number', 'serial_number', 'vendor_sku_barcode',
        'product_category', 'product_status', 'original_qty', 'receive_qty',
        'receive_volume', 'receive_weight', 'quantity_um', 'volume_um',
        'weight_um', 'std_receive_volume', 'std_receive_weight', 'expire_date',
        'create_time', 'receipt_time', 'closed_date', 'last_modified_date',
        'src_inc_day', 'data_source', 'inc_day', 'src'],
       dtype='object')]

In [150]:
?compare_data

Signature:
compare_data(
    df1,
    df2,
    date1,
    date2,
    col1_line,
    col2_line,
    col1_quant,
    col2_quant,
)
Docstring:
数据对比函数, 输出两个df, 一个为全部的对比值, 另一个为有差异的subset
两个df对比(先写bdp的数据), 两个时间戳, 两个line字段名称, 两个quant字段名称
File:      c:\users\dscshap3808\documents\data\cn-214\<ipython-input-128-8060e98b97df>
Type:      function


In [161]:
compare_data(
    fer_inbound_bdp, fer_inbound_ou,
    'closed_date', 'Date',
    'asn_id','Receipt ID',
    'receive_qty', 'Putaway Qty'

)[0]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210602,1.0,1,2380.0,2380.0
1,20210604,4.0,4,4373.0,4373.0
2,20210606,3.0,3,2013.0,2013.0
3,20210607,3.0,3,1523.0,1523.0
4,20210609,2.0,2,308.0,308.0
5,20210610,1.0,1,1.0,1.0
6,20210611,1.0,1,2403.0,2403.0
7,20210613,1.0,1,2529.0,2529.0
8,20210615,10.0,10,4930.0,4930.0
9,20210616,3.0,3,921.0,921.0


---

In [162]:
[fer_outbound_bdp.columns, fer_outbound_ou.columns]

[Index(['wms_company_id', 'wms_company_name', 'wms_warehouse_id',
        'wms_warehouse_name', 'sku_code', 'sku_name', 'sku_desc', 'sku_barcode',
        'batch_number', 'serial_number', 'vendor_sku_barcode', 'order_id',
        'order_line', 'order_status', 'order_type', 'launch_number', 'pick_no',
        'pick_create_date', 'required_ship_date', 'inventory_status',
        'original_order_qty', 'allocated_qty', 'shipped_qty', 'shipped_volume',
        'shipped_weight', 'quantity_um', 'volume_um', 'weight_um',
        'std_shipped_volume', 'std_shipped_weight', 'priority', 'closed_date',
        'create_date', 'last_modified_date', 'src_inc_day', 'data_source',
        'shipment_id', 'inc_day', 'src'],
       dtype='object'),
 Index(['Seq #', 'Post Date', 'Created Date', 'Recv. Date', 'Order No',
        'Ship To Name', 'Location', 'Product Code', 'Description', 'Lot',
        'Ship Qty', 'STR Days', 'Volume', 'Unnamed: 13', 'Unnamed: 14'],
       dtype='object')]

In [165]:
fer_outbound_ou.head(2)

,Seq #,Post Date,Created Date,Recv. Date,Order No,Ship To Name,Location,Product Code,Description,Lot,Ship Qty,STR Days,Volume,Unnamed: 13,Unnamed: 14
0,1.0,2021-06-18 14:15:17.617,2021-06-16 14:00:03.263,2021-06-06 11:03:22.130,1.084917e+09,NSO-成都,G023003,76867709.0,拉斐尔脆雪柔椰蓉扁桃仁糖果酥球原糖285粒装,L039AB154R,47.0,12.0,1.090165,NaN,NaN
1,2.0,2021-06-18 14:15:17.617,2021-06-16 14:00:03.263,2021-06-06 11:03:22.023,1.084917e+09,NSO-成都,G021201,76867709.0,拉斐尔脆雪柔椰蓉扁桃仁糖果酥球原糖285粒装,L039AB154R,50.0,12.0,1.159750,NaN,NaN


In [171]:
compare_data(
    fer_outbound_bdp, fer_outbound_ou,
    'closed_date', 'Post Date',
    'order_id','Order No',
    'shipped_qty', 'Ship Qty'
)[0]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210602,24.0,24,4647.0,4647.0
1,20210603,7.0,7,1616.0,1616.0
2,20210604,14.0,14,1586.0,1586.0
3,20210606,4.0,4,354.0,354.0
4,20210608,11.0,11,2759.0,2759.0
5,20210609,13.0,13,2162.0,2162.0
6,20210610,31.0,31,2071.0,2071.0
7,20210611,12.0,12,1441.0,1441.0
8,20210613,2.0,2,151.0,151.0
9,20210616,8.0,8,568.0,568.0


<span style="color:rgba(255,244,0,1);font-size:42px"><b>COSTA CN-212</b></span> 




In [4]:
cos_inbound_ou = pd.read_excel('./CN-212_COSTA/CN-212 COSTA SHA HT0 WHS.xlsx', sheet_name = 2 ,header = 1)
# cos_outbound_ou = pd.read_excel('./CN-212_COSTA/CN-212 COSTA SHA HT0 WHS.xlsx', sheet_name=  3 )
cos_inbound_bdp =  pd.read_csv('./CN-212_COSTA/inb_costa_bdp.csv') 
# fer_outbound_bdp =  pd.read_csv('./CN-212_COSTA/fe_bdp_oub.csv')  
cos_inb_wmose = pd.read_csv('./CN-212_COSTA/c99_wmose.csv')

In [7]:
cos_inb_wmose

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,internal_id,asn_id,asn_line_id,asn_status,sku_code,sku_name,...,expire_date,create_time,receipt_time,closed_date,last_modified_date,src_inc_day,data_source,is_business_valid,inc_day,src
0,9901497,NaN,C99,NaN,TAOCG202105250002,1747704,11068259,Receiving Started,5014947,costa030400,...,NaN,2021-06-02 15:16:33,2021-06-02 15:38:38,2021-06-02 16:01:03,2021-06-02 15:17:33,20210608,wmos,NaN,20210602,wmos
1,9901497,NaN,C99,NaN,TAOCG202105250002,1747704,11068283,Receiving Started,5014948,costa030401,...,NaN,2021-06-02 15:20:29,2021-06-02 15:38:38,2021-06-02 16:01:03,2021-06-02 15:20:29,20210608,wmos,NaN,20210602,wmos
2,9901497,NaN,C99,NaN,TAOCG202105250002,1747704,11068456,Receiving Started,5014949,costa030459,...,NaN,2021-06-02 15:31:14,2021-06-02 15:38:38,2021-06-02 16:01:03,2021-06-02 15:31:14,20210608,wmos,NaN,20210602,wmos
3,9901497,NaN,C99,NaN,TAOCG202105250002,1747704,11068484,Receiving Started,5014949,costa030459,...,NaN,2021-06-02 15:36:07,2021-06-02 15:38:38,2021-06-02 16:01:03,2021-06-02 15:36:07,20210608,wmos,NaN,20210602,wmos
4,9901497,NaN,C99,NaN,TAOCG202105250002,1747704,11068455,Receiving Started,5014950,costa030458,...,NaN,2021-06-02 15:30:18,2021-06-02 15:38:38,2021-06-02 16:01:03,2021-06-02 15:30:18,20210608,wmos,NaN,20210602,wmos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,9901497,NaN,C99,NaN,TAOCG202106160002,1838228,11388472,Receiving Started,5501216,COSTA030415,...,NaN,2021-06-16 16:28:26,2021-06-17 15:08:22,2021-06-17 15:59:58,2021-06-17 14:52:00,20210623,wmos,NaN,20210617,wmos
57,9901497,NaN,C99,NaN,TAODB202106210002,1859830,11501826,Receiving Started,5014947,costa030400,...,NaN,2021-06-21 14:40:42,2021-06-21 14:57:44,2021-06-21 15:29:14,2021-06-21 14:57:43,20210627,wmos,NaN,20210621,wmos
58,9901497,NaN,C99,NaN,TAODB202106210002,1859830,11502140,Receiving Started,5014948,costa030401,...,NaN,2021-06-21 14:46:10,2021-06-21 14:57:44,2021-06-21 15:29:14,2021-06-21 14:50:40,20210627,wmos,NaN,20210621,wmos
59,9901497,NaN,C99,NaN,TAOCG202106150001,1832913,11707855,Receiving Started,5014958,costa030419,...,NaN,2021-06-30 16:00:15,2021-06-30 16:03:24,2021-06-30 16:10:51,2021-06-30 16:00:15,20210706,wmos,NaN,20210630,wmos


In [6]:
colname_bdp(cos_inb_wmose)
colname_bdp(cos_inbound_bdp)
date_pre(cos_inbound_bdp, ['closed_date'])
date_pre(cos_inbound_ou, ['Actual receiving Date'])

,Warehouse,ASN,Order Date,行号,ASN receiving Date,Actual receiving Date,Supplier,Item Code,Description,ASN数量,Open数量,Stock QTY,单位体积,收货CBM,Base Unit,Stock Specification,状态,短收数量
0,COSTA_SH,02730-20210331-07250,2021-06-02 08:00:00,1,2021-06-02 14:23:47,20210602,NaN,COSTA020730,可乐味硬糖,210,0,210,0.008332,1.749668,盒,1*6*6,Closed,0


In [8]:
date_pre(cos_inb_wmose, ['closed_date'])

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,internal_id,asn_id,asn_line_id,asn_status,sku_code,sku_name,...,expire_date,create_time,receipt_time,closed_date,last_modified_date,src_inc_day,data_source,is_business_valid,inc_day,src
0,9901497,NaN,C99,NaN,TAOCG202105250002,1747704,11068259,Receiving Started,5014947,costa030400,...,NaN,2021-06-02 15:16:33,2021-06-02 15:38:38,20210602,2021-06-02 15:17:33,20210608,wmos,NaN,20210602,wmos


In [9]:
cos_inbound_bdp = cos_inbound_bdp[cos_inbound_bdp['wms_warehouse_id'] == 'COSTA_SH']

In [10]:
[cos_inbound_ou.columns, cos_inbound_bdp.columns]

[Index(['Warehouse', 'ASN', 'Order Date', '行号', 'ASN receiving Date',
        'Actual receiving Date', 'Supplier', 'Item Code', 'Description',
        'ASN数量', 'Open数量', 'Stock QTY', '单位体积', '收货CBM', 'Base Unit',
        'Stock Specification', '状态', '短收数量'],
       dtype='object'),
 Index(['wms_company_id', 'wms_company_name', 'wms_warehouse_id',
        'wms_warehouse_name', 'internal_id', 'asn_id', 'asn_line_id',
        'asn_status', 'sku_code', 'sku_name', 'sku_desc', 'sku_barcode',
        'batch_number', 'serial_number', 'vendor_sku_barcode',
        'product_category', 'product_status', 'original_qty', 'receive_qty',
        'receive_volume', 'receive_weight', 'quantity_um', 'volume_um',
        'weight_um', 'std_receive_volume', 'std_receive_weight', 'expire_date',
        'create_time', 'receipt_time', 'closed_date', 'last_modified_date',
        'src_inc_day', 'data_source', 'is_business_valid', 'inc_day', 'src'],
       dtype='object')]

In [11]:
?compare_data

Signature:
compare_data(
    df1,
    df2,
    date1,
    date2,
    col1_line,
    col2_line,
    col1_quant,
    col2_quant,
)
Docstring:
数据对比函数, 输出两个df, 一个为全部的对比值, 另一个为有差异的subset
两个df对比(先写bdp的数据), 两个时间戳, 两个line字段名称, 两个quant字段名称
File:      c:\users\dscshap3808\documents\data\cn-214\<ipython-input-3-b027b23cf1da>
Type:      function


In [24]:
compare_data( # scale vs ou
    cos_inbound_bdp, cos_inbound_ou,
    'closed_date', 'Actual receiving Date',
    'asn_id', 'ASN',
    'receive_qty','Stock QTY'
)[0]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,5,5.0,10860.0,10860.0
1,20210602,6,4.0,7870.0,5470.0
2,20210603,18,14.0,17949.0,12845.0
3,20210604,4,1.0,3400.0,100.0
4,20210606,2,2.0,9098.0,9098.0
5,20210607,4,1.0,13157.0,449.0
6,20210608,3,2.0,50522.0,50512.0
7,20210609,1,NaN,11484.0,NaN
8,20210610,28,23.0,36493.0,14040.0
9,20210611,2,2.0,519.0,519.0


In [16]:
compare_data( # scale vs wmos
    cos_inbound_bdp, cos_inb_wmose,
    'closed_date', 'closed_date',
    'asn_id', 'asn_id',
    'receive_qty','receive_qty'
)[0]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,5,NaN,NaN,NaN
1,20210602,6,4.0,7870.0,7985.0
2,20210603,18,NaN,NaN,NaN
3,20210604,4,NaN,NaN,NaN
4,20210606,2,NaN,NaN,NaN
5,20210607,4,NaN,NaN,NaN
6,20210608,3,NaN,NaN,NaN
7,20210609,1,NaN,NaN,NaN
8,20210610,28,1.0,36493.0,50.0
9,20210611,2,4.0,519.0,8904.0


In [25]:
compare_data( # scale vs wmos
    cos_inbound_bdp, cos_inb_wmose,
    'closed_date', 'closed_date',
    'asn_id', 'asn_id',
    'receive_qty','receive_qty'

)[1]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
1,20210602,6,4.0,7870.0,7985.0
8,20210610,28,1.0,36493.0,50.0
9,20210611,2,4.0,519.0,8904.0
12,20210616,8,1.0,17761.0,10416.0
13,20210617,9,1.0,39644.0,104.0
17,20210621,5,1.0,19344.0,19561.0
24,20210630,25,1.0,156994.0,4360.0


In [26]:
compare_data( # wmos vs ou
    cos_inb_wmose, cos_inbound_ou,
    'closed_date', 'Actual receiving Date',
    'asn_id', 'ASN',
    'receive_qty','Stock QTY'

)[0]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210602,4.0,4,7985.0,5470
1,20210610,1.0,23,50.0,14040
2,20210611,4.0,2,8904.0,519
3,20210616,1.0,6,10416.0,14457
4,20210617,1.0,9,104.0,39644
5,20210621,1.0,1,19561.0,4
6,20210630,1.0,7,4360.0,2652
7,20210601,NaN,5,NaN,10860
8,20210603,NaN,14,NaN,12845
9,20210604,NaN,1,NaN,100


In [13]:
?detail_check

Signature: detail_check(inb_bose_bdp, inb_bose_oud, wrong_time, cols_bdp, cols_ou)
Docstring:
输入两个表名字后, 写有问题的时间查看细节区别,
cols = id, qty, date
File:      c:\users\dscshap3808\documents\data\cn-214\<ipython-input-3-b027b23cf1da>
Type:      function


wmos


In [38]:
cos_inb_wmose['asn_id'] = cos_inb_wmose['asn_id'].astype(str)
cos_inb_wmose['sku_name'] = cos_inb_wmose['sku_name'].str.upper()

In [70]:
cos_inbound_bdp

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,internal_id,asn_id,asn_line_id,asn_status,sku_code,sku_name,...,expire_date,create_time,receipt_time,closed_date,last_modified_date,src_inc_day,data_source,is_business_valid,inc_day,src
151,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11098,01-20210426-07521,64932,Closed,COSTA030574,COSTA030574,...,NaN,2021-04-26 08:01:02,2021-05-05 00:00:00,20210601,2021-06-01 02:49:55,20210811,scale_costacoffee,1,20210601,scale
152,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11121,01-20210429-07557,65022,Closed,COSTA060438,COSTA060438,...,NaN,2021-04-29 10:01:06,2021-05-05 00:00:00,20210601,2021-06-01 06:47:06,20210811,scale_costacoffee,1,20210601,scale
153,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11250,01-20210520-07746,65590,Closed,COSTA050333,COSTA050333,...,NaN,2021-05-20 09:01:10,2021-05-26 00:00:00,20210601,2021-06-01 06:46:50,20210811,scale_costacoffee,1,20210601,scale
154,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11305,01-20210527-07843,66016,Closed,COSTA040089,COSTA040089,...,NaN,2021-05-27 09:16:05,2021-06-02 00:00:00,20210601,2021-06-01 09:25:45,20210811,scale_costacoffee,1,20210601,scale
155,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11318,01-20210527-07856,66058,Closed,COSTA040358,COSTA040358,...,NaN,2021-05-27 09:16:08,2021-06-02 00:00:00,20210601,2021-06-01 09:25:30,20210811,scale_costacoffee,1,20210601,scale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11508,01-20210625-08108,67034,Check In Pending,COSTA011811,COSTA011811,...,NaN,2021-06-25 08:01:02,2021-06-30 00:00:00,20210701,2021-07-01 11:24:58,20210811,scale_costacoffee,1,20210701,scale
1012,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11508,01-20210625-08108,67043,Check In Pending,COSTA011861,COSTA011861,...,NaN,2021-06-25 08:01:02,2021-06-30 00:00:00,20210701,2021-07-01 11:24:58,20210811,scale_costacoffee,1,20210701,scale
1013,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11520,01-20210630-08132,67090,Closed,COSTA040111,COSTA040111,...,NaN,2021-06-30 07:01:01,2021-07-01 00:00:00,20210701,2021-07-01 10:38:39,20210811,scale_costacoffee,1,20210701,scale
1014,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11524,01-20210701-08139,67097,Closed,COSTA050474,COSTA050474,...,2022-05-31 00:00:00,2021-07-01 09:01:00,2021-07-02 00:00:00,20210701,2021-07-01 09:25:02,20210811,scale_costacoffee,1,20210701,scale


In [73]:
cos_inb_wmose[['sku_name', 'receive_qty', 'closed_date']].groupby()

,sku_name,receive_qty,closed_date
0,COSTA030400,1896.0,20210602
1,COSTA030401,1080.0,20210602
2,COSTA030459,315.0,20210602
3,COSTA030459,325.0,20210602
4,COSTA030458,640.0,20210602
...,...,...,...
56,COSTA030415,14.0,20210617
57,COSTA030400,15042.0,20210621
58,COSTA030401,4519.0,20210621
59,COSTA030419,80.0,20210630


In [76]:
detail_check(
    cos_inbound_bdp, cos_inb_wmose, '20210602', 
    cols_bdp = ['sku_name', 'receive_qty', 'closed_date'], 
    cols_ou = ['sku_name', 'receive_qty', 'closed_date'])


ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [41]:
detail_check(
     cos_inbound_bdp, cos_inbound_ou,
     '20210604',
    cols_bdp = ['asn_id', 'receive_qty', 'closed_date'], 
    cols_ou = ['ASN','Stock QTY', 'Actual receiving Date']
)[0]


,bdp_id,bdp_qty,ou_id,ou_qty
0,01-20210429-07576,100.0,01-20210429-07576,100.0
1,01-20210527-07857,3000.0,NaN,NaN
2,01-20210603-07895,0.0,NaN,NaN
3,05-20210603,300.0,NaN,NaN


In [31]:
time_all = compare_data(
    cos_inbound_bdp, cos_inbound_ou,
    'closed_date', 'Actual receiving Date',
    'asn_id', 'ASN',
    'receive_qty','Stock QTY'

)[1]['date_time']

In [42]:
checkl_detial =pd.DataFrame()
for i in time_all:
    checkl_detial = pd.concat([checkl_detial,
        detail_check(
            cos_inbound_bdp, cos_inbound_ou, i,
            cols_bdp = ['asn_id', 'receive_qty', 'closed_date'], 
            cols_ou = ['ASN','Stock QTY', 'Actual receiving Date'])[0]], axis = 0)

In [47]:
checkl_detial.dropna(how = 'any', axis = 1).to_csv('./CN-212_COSTA/chekc_detail_all.csv', index=False)

In [20]:
cos_inbound_bdp[cos_inbound_bdp['closed_date'] == '20210602']['asn_id'].unique()

array(['01-20210520-07753', '01-20210527-07835', '01-20210527-07862',
       '01-20210601-07880', '01-20210602-07885', '02730-20210331-07250'],
      dtype=object)

In [21]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)

cos_inbound_bdp[cos_inbound_bdp['asn_id'] =='01-20210520-07753']

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,internal_id,asn_id,asn_line_id,asn_status,sku_code,sku_name,sku_desc,sku_barcode,batch_number,serial_number,vendor_sku_barcode,product_category,product_status,original_qty,receive_qty,receive_volume,receive_weight,quantity_um,volume_um,weight_um,std_receive_volume,std_receive_weight,expire_date,create_time,receipt_time,closed_date,last_modified_date,src_inc_day,data_source,is_business_valid,inc_day,src
156,CostaCoffee,CostaCoffee,COSTA_SH,COSTA_SH,11257,01-20210520-07753,65624,Closed,COSTA020671,COSTA020671,蜂蜜燕麦希腊式酸奶杯,NaN,NaN,NaN,NaN,冷藏,NaN,400.0,400.0,0.0,0.0,个,NaN,NaN,NaN,NaN,NaN,2021-05-20 09:01:12,2021-05-26 00:00:00,20210602,2021-06-02 01:34:34,20210811,scale_costacoffee,1,20210602,scale


In [22]:
cos_inbound_ou[cos_inbound_ou['ASN'] =='01-20210520-07753']

,Warehouse,ASN,Order Date,行号,ASN receiving Date,Actual receiving Date,Supplier,Item Code,Description,ASN数量,Open数量,Stock QTY,单位体积,收货CBM,Base Unit,Stock Specification,状态,短收数量


<span style="color:rgba(255,244,0,1);font-size:42px"><b>FUJI CKG CN-177</b></span> 




In [126]:
-- select internal_id, asn_id,asn_line_id,sku_code,receive_qty,create_time,receipt_time,closed_date
-- from  dsc_dwd.dwd_wh_dsc_inbound_line_dtl_di
-- where src = 'scale'
-- and inc_day between '20210601' and '20210701'
-- and data_source = 'scale_fuji'
-- and wms_warehouse_id = 'FUJICKG'

SyntaxError: invalid syntax (<ipython-input-126-f0a1acf4a7c9>, line 1)

In [155]:
fuji_inbound_ou = pd.read_excel(
    './CN-177_FUJI/Inbound.xlsx', 
    header=1)
    
fuji_outbound_ou = pd.read_excel(
    './CN-177_FUJI/Outbound.xlsx', 
    header=1)
    
fuji_inbound_bdp =  pd.read_csv('./CN-177_FUJI/fuji_in.csv')  
fuji_oubound_bdp =  pd.read_csv('./CN-177_FUJI/out_fuji.csv')  
[fuji_inbound_ou.columns, fuji_outbound_ou.columns]

 

In [161]:
# colname_bdp(fuji_oubound_bdp)

In [158]:
# colname_bdp(cos_inbound_bdp)
date_pre(fuji_inbound_bdp, ['closed_date'])
date_pre(fuji_inbound_ou, ['Receipt Date'])
date_pre(fuji_oubound_bdp, ['closed_date'])
date_pre(fuji_outbound_ou, ['Shipped Date'])

,Req,Warehouse,Customer PO,Shipment ID,Order\nType,Process Type,Freight Terms,ERP Order,Line Num,Item,Item Desc,Requested Qty,Total Qty,Total Volume,Total Weight,Ship To Id,Ship To Name,Ship To Address,City,Postal Code,Create Date,Shipped Date,POD Date Time,Company,Leading Status,Trailing Status,item_length,item_width,item_height
0,1,FUJICKG,50011622,ID6810301.1,FXID_FMEQ,NaN,NaN,50011622,1,EC103908,EC103908,1,1,0.368,0.05,CN010800567,青岛施乐办公设备有限公司,重庆市江北区港城南路一号海尔工业园,重庆市,NaN,2021-06-15 15:07:02.160,20210615,NaN,FXID,Closed,Closed,2.3,1.6,0.1


In [159]:
# 筛选条件
fuji_inbound_bdp = fuji_inbound_bdp[~fuji_inbound_bdp['asn_id'].str.contains('R')]
fuji_inbound_bdp = fuji_inbound_bdp[~fuji_inbound_bdp['asn_id'].str.contains('旧件')]

In [162]:
compare_data(
    fuji_inbound_bdp, fuji_inbound_ou,
    'closed_date', 'Receipt Date',
    'asn_id', 'Receipt Id',
    'receive_qty','Received Qty'

)[0]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,3,3,718.0,718
1,20210604,1,1,1.0,1
2,20210607,10,10,685.0,685
3,20210608,3,3,6.0,6
4,20210610,1,1,1.0,1
5,20210611,2,2,6.0,6
6,20210615,3,3,38.0,38
7,20210616,4,4,422.0,422
8,20210617,5,5,10.0,10
9,20210618,1,1,9.0,9


In [116]:
compare_data(
    fuji_inbound_bdp, fuji_inbound_ou,
    'closed_date', 'Receipt Date',
    'asn_id', 'Receipt Id',
    'receive_qty','Received Qty'

)[1]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty


In [117]:
detail_check(
     fuji_inbound_bdp, fuji_inbound_ou,
     '20210629',
    cols_bdp = ['asn_id', 'receive_qty', 'closed_date'], 
    cols_ou = ['Receipt Id','Received Qty', 'Receipt Date']
)[0]


,bdp_id,bdp_qty,ou_id,ou_qty
0,104729024新件,1.0,104729024新件,1
1,83410986退新件,1.0,83410986退新件,1
2,ID6970967,4.0,ID6970967,4


> outbound

In [118]:
fuji_outbound_ou.columns

Index(['Req', 'Warehouse', 'Customer PO', 'Shipment ID', 'Order\nType',
       'Process Type', 'Freight Terms', 'ERP Order', 'Line Num', 'Item',
       'Item Desc', 'Requested Qty', 'Total Qty', 'Total Volume',
       'Total Weight', 'Ship To Id', 'Ship To Name', 'Ship To Address', 'City',
       'Postal Code', 'Create Date', 'Shipped Date', 'POD Date Time',
       'Company', 'Leading Status', 'Trailing Status', 'item_length',
       'item_width', 'item_height'],
      dtype='object')

In [75]:
-- set hive.support.quoted.identifiers=none;
-- select  
-- `(sku_code|sku_name)?+.+`
--  from  dsc_dwd.dwd_wh_dsc_outbound_line_dtl_di
-- where src = 'scale'
-- and inc_day between '20210601' and '20210701'
-- and data_source = 'scale_fuji'
-- and wms_warehouse_id = 'FUJICKG'

SyntaxError: invalid syntax (<ipython-input-75-8dd198250478>, line 1)

In [125]:
fuji_oubound_bdp

,wms_warehouse_id,sku_code,order_id,order_line,closed_date,create_date,last_modified_date
0,FUJICKG,CWAA0751,ID6923388,730328,20210601,2021-06-01 05:07:02,2021-06-01 05:10:59
1,FUJICKG,CWAA0952,ID6923403,730318,20210601,2021-06-01 05:07:02,2021-06-01 05:10:59
2,FUJICKG,CT202638C,ID6923403,730317,20210601,2021-06-01 05:07:02,2021-06-01 05:10:59
3,FUJICKG,CT202208,ID6923404,730329,20210601,2021-06-01 05:07:02,2021-06-01 05:10:59
4,FUJICKG,CT350935,ID6923439,730344,20210601,2021-06-01 05:07:03,2021-06-01 05:10:59
...,...,...,...,...,...,...,...
2107,FUJICKG,CT202344,ID6983408,738334,20210701,2021-07-01 09:04:02,2021-07-01 09:35:42
2108,FUJICKG,CWAA0900,ID6983420,738332,20210701,2021-07-01 09:04:01,2021-07-01 09:35:42
2109,FUJICKG,CT202103,ID6983584,738344,20210701,2021-07-01 09:34:01,2021-07-01 09:35:42
2110,FUJICKG,CT202104,ID6983584,738345,20210701,2021-07-01 09:34:01,2021-07-01 09:35:42


In [164]:
# compare_data(
#     fer_outbound_bdp, fer_outbound_ou,
#     'closed_date', 'Post Date',
#     'order_id','Order No',
#     'shipped_qty', 'Ship Qty'
# )[0]
pd.set_option("display.max_rows", None, "display.max_columns", None)
compare_data(
    fuji_oubound_bdp, fuji_outbound_ou,
    'closed_date', 'Shipped Date',
    'order_id', 'Shipment ID',
    'shipped_qty','Total Qty'

)[0] 

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty
0,20210601,72,72,165.0,165
1,20210602,51,51,105.0,105
2,20210603,42,42,91.0,91
3,20210604,55,55,126.0,126
4,20210607,60,60,148.0,148
5,20210608,17,17,49.0,49
6,20210609,92,92,253.0,253
7,20210610,38,38,92.0,92
8,20210611,50,50,100.0,100
9,20210615,68,68,152.0,152


In [165]:
compare_data(
    fuji_oubound_bdp, fuji_outbound_ou,
    'closed_date', 'Shipped Date',
    'order_id', 'Shipment ID',
    'shipped_qty','Total Qty'

)[1]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty


<span style="color:rgba(255,244,0,1);font-size:42px"><b>SQUIBB CN-009</b></span> 




In [ ]:
 select * from  dsc_dwd.dwd_wh_dsc_outbound_line_dtl_di
 where inc_day between '20210601' and '20210701'
 and src = 'scale'
 and data_source = 'scale_squibb'

In [90]:
squibb_inbound_ou = pd.read_excel(
    './CN-009_squibb/Data Validation_15072021.xlsx', 
    header=0, sheet_name = 2)
    
squibb_outbound_ou = pd.read_excel(
    './CN-009_squibb/Data Validation_15072021.xlsx', 
    header=0, sheet_name = 3 )
    
squibb_inbound_bdp =  pd.read_csv('./CN-009_squibb/squibb_in.csv')  
squibb_oubound_bdp =  pd.read_csv('./CN-009_squibb/squibb_out.csv')  
[squibb_inbound_ou.columns, squibb_outbound_ou.columns]



[Index(['ASN', 'SKU', 'LOT', 'QTY', 'UM', 'DATE'], dtype='object'),
 Index(['PKT', 'SKU', 'LOT', 'QTY', 'UM', 'DATE'], dtype='object')]

In [91]:
# squibb_inbound_ou['DATE'] = squibb_inbound_ou['DATE']+ pd.Timedelta(hours=8) 
# squibb_outbound_ou['DATE'] = squibb_outbound_ou['DATE']+ pd.Timedelta(hours=8) 


In [92]:
colname_bdp(squibb_inbound_bdp)
colname_bdp(squibb_oubound_bdp)


# colname_bdp(cos_inbound_bdp)
date_pre(squibb_inbound_bdp, ['create_time'])

date_pre(squibb_inbound_ou, ['DATE'])
date_pre(squibb_oubound_bdp, ['create_date'])
date_pre(squibb_oubound_bdp, ['closed_date'])

date_pre(squibb_outbound_ou, ['DATE'])

,PKT,SKU,LOT,QTY,UM,DATE
0,5189,1165801,ABV6038,700,EA,20210602


In [110]:
compare_data(
    squibb_inbound_bdp, squibb_inbound_ou,
    'create_time', 'DATE',
    'asn_id', 'ASN',
    'receive_qty','QTY'

)[0].sum()

date_time     2021060120210602202106032021060720210608202106...
bdp_hdr_ct                                                   38
ou_hdr_ct                                                    38
bdp_qty                                               1120530.0
ou_qty                                                  1120530
dtype: object

In [54]:
compare_data(
    squibb_inbound_bdp, squibb_inbound_ou,
    'create_time', 'DATE',
    'asn_id', 'ASN',
    'receive_qty','QTY'

)[1]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty


> outbound

In [109]:
compare_data(
   squibb_oubound_bdp, squibb_outbound_ou,
    'create_date', 'DATE',
    'order_id', 'PKT',
    'shipped_qty','QTY'

)[0].sum()

date_time     2021060220210607202106082021060920210611202106...
bdp_hdr_ct                                                   17
ou_hdr_ct                                                    17
bdp_qty                                                369240.0
ou_qty                                                   369240
dtype: object

In [103]:
compare_data(
   squibb_oubound_bdp, squibb_outbound_ou,
    'create_date', 'DATE',
    'order_id', 'PKT',
    'shipped_qty','QTY'

)[1]

,date_time,bdp_hdr_ct,ou_hdr_ct,bdp_qty,ou_qty


In [104]:

detail_check(
     squibb_oubound_bdp, squibb_outbound_ou,
     '20210608',
    cols_bdp = ['order_id', 'shipped_qty', 'create_date'], 
    cols_ou = ['PKT','QTY', 'DATE']
)[0]


,bdp_id,bdp_qty,ou_id,ou_qty
0,5191,300.0,5191,300


In [105]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)
detail_check(
     squibb_oubound_bdp, squibb_outbound_ou,
     '20210609',
    cols_bdp = ['order_id', 'shipped_qty', 'create_date'], 
    cols_ou = ['PKT','QTY', 'DATE']
)[0]


,bdp_id,bdp_qty,ou_id,ou_qty
0,8016297687,24800.0,8016297687,24800
1,8016298000,4800.0,8016298000,4800
2,8016298001,60000.0,8016298001,60000


In [106]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)
detail_check(
     squibb_oubound_bdp, squibb_outbound_ou,
     '20210610',
    cols_bdp = ['order_id', 'shipped_qty', 'create_date'], 
    cols_ou = ['PKT','QTY', 'DATE']
)[0]


,bdp_id,bdp_qty,ou_id,ou_qty
